<a href="https://colab.research.google.com/github/indra622/AIAcademy_SpeechRecognition/blob/main/2_dataset_and_MLP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기초 MLP 모델 실습



## Torch Dataset

git  cloning  : 
`!git clone https://github.com/Jakobovski/free-spoken-digit-dataset`

In [ ]:
!git clone https://github.com/Jakobovski/free-spoken-digit-dataset


fatal: destination path 'free-spoken-digit-dataset' already exists and is not an empty directory.


In [ ]:
import torch
import torchaudio
from pathlib import Path


## PyTorch Daset Object 개념 
`torch.utils.data.Dataset`의 subclass는 `__len__`과 `__getitem__`을 구현하기 위해 필요한 객체이다
- `__len__` : 데이터셋의 아이템 수를 반환
- `__getitem__`: 샘플과 레이블을 반환


[그림]데이터를 직접적으로 가지고 있지 않지만 `__len__` 과 `__getitem__`을 통해 접근가능

<img src="https://drek4537l1klr.cloudfront.net/stevens2/Figures/CH07_F02_Stevens2_GS.png" width=400>

In [ ]:
from torch.utils.data import Dataset,DataLoader

## Dataset Calss 의 형식

아래와 같이 3개의 method를 override하여 작성

```python
class MyCustomDataset(Dataset):
    def __init__(self, ...):
        # initialize Class
        
    def __getitem__(self, index):
        # return samlpe(data) and label 
        return (img, label)

    def __len__(self):
      # of how many examples(images?) you have
        return count 
```

In [ ]:
class AudioDataset(Dataset):
  def __init__(self,dir="./free-spoken-digit-dataset/recordings"):
    self.audio_list = list(Path(dir).rglob("*.wav"))

  def __len__(self):
    return len(self.audio_list)
  
  def __getitem__(self,idx):
    return self.audio_list[idx]

In [ ]:
train = AudioDataset()

In [ ]:
train[:10]

[PosixPath('free-spoken-digit-dataset/recordings/7_theo_28.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/0_yweweler_7.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/6_yweweler_47.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/5_lucas_36.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/7_jackson_30.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/7_nicolas_3.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/3_george_2.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/0_nicolas_0.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/8_george_2.wav'),
 PosixPath('free-spoken-digit-dataset/recordings/8_nicolas_34.wav')]

In [ ]:
bs=64
train_loader = DataLoader(train,batch_size=bs)

In [ ]:
for _ in train_loader:
  pass

TypeError: ignored

In [ ]:
class AudioDataset(Dataset):
  def __init__(self,dir="./free-spoken-digit-dataset/recordings"):
    self.audio_list = list(Path(dir).rglob("*.wav"))
    self.sr=8000
    

  def __len__(self):
    return len(self.audio_list)
  
  def __getitem__(self,idx):
    audio_path = self.audio_list[idx]
    y,sr = torchaudio.load(audio_path)
    y = y.mean(dim=0)
    label =str(audio_path.name).split("_")[0]
    return y,label

In [ ]:
train = AudioDataset()
train[0]

In [ ]:
import IPython.display as ipd
data,label = train[0]
print(label)
ipd.Audio(data,rate=train.sr)

In [ ]:
train_loader = DataLoader(train,batch_size=bs)

In [ ]:
for batch in train_loader:
  print(batch.shape)

In [ ]:
class AudioDataset(Dataset):
  def __init__(self,dir="./free-spoken-digit-dataset/recordings"):
    self.audio_list = list(Path(dir).rglob("*.wav"))
    self.sr=8000
  def __len__(self):
    return len(self.audio_list)
  
  def __getitem__(self,idx):
    audio_path = self.audio_list[idx]
    y,sr = torchaudio.load(audio_path)
    padding = torch.zeros(20000)
    y = y.mean(dim=0)
    padding[:len(y)]=y
    label =str(audio_path.name).split("_")[0]
    return padding,int(label)

In [ ]:
train = AudioDataset()
train[0]

In [ ]:

train_loader = DataLoader(train,batch_size=bs)

In [ ]:
for batch in train_loader:
  pass  

# Split Dataset
train, valid

파일 이름을 이용해서 train/test를 split

In [ ]:
audio_list = list(Path("./free-spoken-digit-dataset/recordings").rglob("*.wav"))
print(audio_list[0])
print(audio_list[0].stem)
print(audio_list[0].stem.split("_")[-1])

In [ ]:
class AudioDataset(Dataset):
  def __init__(self,dir="./free-spoken-digit-dataset/recordings",train=True):
    self.audio_list = list(Path(dir).rglob("*.wav"))
    self.sr=8000
    self.train_list,self.test_list = self._split_data()
    if train:
      self.audio_list = self.train_list
    else :
      self.audio_list = self.test_list
  
  def _split_data(self) : 
    """
    The test set officially consists of the first 10% of the recordings. 
    Recordings numbered 0-4 (inclusive) are in the test and 5-49 are in the training set.
    """
    testsplit = ['0','1','2','3','4']
    train_list = []
    test_list = []
    for audio_path in self.audio_list:
      
      recording_number = audio_path.stem.split("_")[-1] #if 
      if recording_number in testsplit:
        test_list.append(audio_path)
      else:
        train_list.append(audio_path)
    return train_list,test_list


  def __len__(self):
    return len(self.audio_list)
  
  def __getitem__(self,idx):
    audio_path = self.audio_list[idx]
    y,sr = torchaudio.load(audio_path)
    padding = torch.zeros(20000)
    y = y.mean(dim=0)
    padding[:len(y)]=y
    padding = padding[:8000]
    label =str(audio_path.name).split("_")[0]
    return padding,int(label)

In [ ]:
train = AudioDataset(train=True)
test = AudioDataset(train=False)

In [ ]:
len(train),len(test)

In [ ]:
def valid_model(model,data_loader,device,loss_fn):
  val_loss=0
  val_acc = 0
  model.eval()
  with torch.no_grad():
    for batch in data_loader:
      x,y = batch
      x = x.to(device)
      y= y.to(device)    
      pred = model(x)
      loss = loss_fn(pred,y)
      val_loss += loss.cpu().item()*len(y)
      val_acc +=(torch.argmax(pred.cpu(),dim=-1) == y).sum(float).item()
    val_loss /=len(data_loader.dataset)
    val_acc /=len(data_loader.dataset)
    return val_loss,val_acc

## Model 정의
`torch.nn`을 import

`torch.nn.Module` 의 subclass로 절의

In [ ]:
import torch.nn as nn
import torch

In [ ]:
from math import ceil 

class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.n_mels = 64
    self.n_fft = 256
    self.hop_length=self.n_fft//2
    self.sr = 8000
    self.n_frames = ceil(8000/self.hop_length)  
     #input size = n_mels*n_frames, 64*79
    self.n_classes = 10
    self.hidden_dim = 1024
    self.mel_converter = torchaudio.transforms.MelSpectrogram(sample_rate=self.sr,
                                                              n_fft=self.n_fft,
                                                              hop_length=self.hop_length,
                                                              n_mels=self.n_mels)
    self.db_converter = torchaudio.transforms.AmplitudeToDB()

    self.nn_layer = nn.Sequential(
                          nn.Linear(self.n_mels*self.n_frames,self.hidden_dim),
                          nn.ReLU(),
                          nn.Linear(self.hidden_dim,self.hidden_dim),                                   
                          nn.ReLU(),
                          nn.Linear(self.hidden_dim,self.hidden_dim),                                   
                          nn.ReLU(),
                          nn.Linear(self.hidden_dim,self.hidden_dim),                                   
                          nn.ReLU(),
                          nn.Linear(self.hidden_dim,self.n_classes)
    )

  def forward(self,x):
    #print(x.shape)
    spec = self.mel_converter(x)
    #print(spec.shape)
    if spec.dim() >2:
      spec = spec.reshape(spec.shape[0],-1)
    else :
      spec = spec.flatten()
    #print(spec.shape)
    spec = self.db_converter(spec)
    #print(spec.shape)
    x = self.nn_layer(spec)
    
    return x


      

In [ ]:
model = MyModel()

In [ ]:
x,label = train[0]
out = model(x)

In [ ]:
out

In [ ]:
out.argmax()

In [ ]:
  probs = torch.softmax(out,dim=-1)
  probs

In [ ]:
probs.sum()

In [ ]:
bs=16
train_loader = DataLoader(train,batch_size=bs,shuffle=True)

In [ ]:
batch  = next(iter(train_loader))

In [ ]:
x,y = batch

In [ ]:
y

In [ ]:
outs =model(x)

In [ ]:
print(outs.shape)
print(outs)

In [ ]:
outs.argmax(dim=-1)

## Train Loop

In [ ]:
from tqdm.auto import tqdm

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
model = MyModel()
xent = nn.CrossEntropyLoss()
model



In [ ]:
torch.manual_seed(0)
pred = torch.softmax(torch.rand(4,10),dim=-1)
pred, pred.argmax(dim=-1)

In [ ]:
target= torch.tensor([7,7,6,1])
xent(pred,target)


In [ ]:
pred = torch.zeros(4,10)
pred[0,7]=99999
pred[1,7]=99999
pred[2,6]=99999
pred[3,1]=99999


In [ ]:
print(pred)

In [ ]:
softmax_pred=pred.softmax(dim=-1)
softmax_pred, pred.argmax(dim=-1)

In [ ]:
xent(pred,target)


In [ ]:
print(next(model.parameters()).device)
model.to(device)
print(next(model.parameters()).device)

In [ ]:
label = []
for batch in train_loader:
  x,y = batch
  label.extend(y.tolist())

In [ ]:
from collections import Counter
Counter(label)

In [ ]:
def valid_model(model,data_loader,device,loss_fn):
  val_loss=0
  val_acc = 0
  model.eval()
  with torch.no_grad():
    for batch in data_loader:
      x,y = batch
      x = x.to(device)
      y= y.to(device)    
      pred = model(x)
      loss = loss_fn(pred,y)
      val_loss += loss.cpu().item()*len(y)
      val_acc +=(torch.argmax(pred,dim=-1) == y).sum().float().item()
    val_loss /=len(data_loader.dataset)
    val_acc /=len(data_loader.dataset)
  model.train()
  return val_loss,val_acc

In [ ]:
num_epochs = 200
bs=128
losses = []
val_losses = []
val_accs = []
model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.0001)

train_loader = DataLoader(train,batch_size=bs,shuffle=True)
test_loader = DataLoader(test,batch_size=bs,shuffle=False)

loss_fn = nn.CrossEntropyLoss()
pbar =  tqdm(range(num_epochs))
for epoch in pbar:
  #for batch in tqdm(train_loader,leave=False):
  for batch in train_loader:
    signal,label = batch
    signal = signal.to(device)
    label = label.to(device)
    pred = model(signal)
    #pred= pred.argmax(dim=-1)
    
    #print(pred.device)
    loss = loss_fn(pred,label.to(device))
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss.item())
  val_loss,val_acc = valid_model(model,test_loader,'cuda',loss_fn)
  val_losses.append(val_loss)
  val_accs.append(val_acc)
  pbar.set_postfix({"train_loss":loss.item(),"val_loss":val_loss,"val_acc":val_acc})
  
  #print(val_losses[-1],val_accs[-1])


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(losses)
plt.show()
plt.plot(val_losses)
plt.show()
plt.plot(val_accs)
plt.show()

In [ ]:
min(losses)

In [ ]:
testing_idx = 80
model.to('cpu')
model.eval()
with torch.no_grad() : 
  x,label =train[testing_idx]
  pred = model(x)
  


In [ ]:
torch.argmax(torch.softmax(pred,dim=-1)),label